In [1]:
import numpy as np
import pandas as pd
# import sklearn as sk
import plotly.graph_objects as go
import plotly.express as px


The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


In [2]:
import sys
sys.path.append('C:\\Users\\filip\meu_computador\\Github\\repositories\\kaggle\\kaggle_challenges\\doordash_challenge')
from doordash_challenge.models.functions.Cleaner import DataCleaner
from doordash_challenge.models.functions.Handler import DataHandler

# Reading data

In [3]:
data = pd.read_csv('C:\\Users\\filip\\meu_computador\\Github\\repositories\\kaggle\\kaggle_challenges\\doordash_challenge\\data\\historical_data.csv')
data['created_at'] = pd.to_datetime(data['created_at'])
data['actual_delivery_time'] = pd.to_datetime(data['actual_delivery_time'])
data  = data.sort_values('created_at')
data = data.drop(['min_item_price', 'max_item_price'],axis=1)

## Simple Cleaning

In [4]:
data_clean_temporal = DataCleaner.add_temporal_variables(data)
data_clean_target = DataCleaner.add_target_variables(data_clean_temporal.copy(), delivery_threshold=3)
data_cleaning = DataCleaner.remove_negative_values(data_clean_target.copy())


In [5]:
# filling categorinal nan values 
whole_data = data_cleaning.copy()
whole_data['market_id'] = whole_data['market_id'].fillna('not informed')
whole_data['market_id'] = whole_data['market_id'].astype(str)

whole_data['order_protocol'] = whole_data['order_protocol'].fillna('not informed')
whole_data['order_protocol'] = whole_data['order_protocol'].astype(str)

whole_data['store_primary_category'] = whole_data['store_primary_category'].fillna('not informed')


## Input missing data of categorical columns

In [6]:
fillna_datahandler = DataHandler(data=data_cleaning.copy(),column_methods={'market_id':'fill with new category', 'store_primary_category':'fill with new category'})
data_cleaning_no_categorical_na =  fillna_datahandler.fill_na_with_new_category()

In [7]:
data_cleaning_no_categorical_na.shape

(197202, 21)

## Correction of the whole data for EDA

In [8]:
curated_data = DataCleaner.clean_conflict_category(data_cleaning_no_categorical_na.copy())

In [9]:
data_cleaning_no_categorical_na.loc[data_cleaning_no_categorical_na['store_id'].eq(4210)]['market_id'].value_counts()

market_id
2.0             474
4.0               4
1.0               4
3.0               2
5.0               2
not informed      2
6.0               2
Name: count, dtype: int64

In [10]:
curated_data.loc[curated_data['store_id'].eq(4210)]['market_id'].value_counts()

market_id
2.0    490
Name: count, dtype: int64

In [11]:
column_methods={
    'market_id':'fill with new category', 
    'store_primary_category':'fill with new category',
    'order_protocol':'fill with median',
    'total_outstanding_orders':'fill with cluster median',
    'total_onshift_dashers':'fill with cluster median',
    'availabe_dashers_onshift':'fill with cluster median'
}

fillna_datahandler = DataHandler(data=curated_data.copy(),column_methods=column_methods)
curated_data_complete =  fillna_datahandler.fill_na_with_new_category()
curated_data_complete =  fillna_datahandler.fill_na_with_median()
curated_data_complete =  fillna_datahandler.fill_na_with_cluster_median()

In [12]:
curated_data_complete.shape

(197202, 21)

In [13]:
fillna_datahandler.fill_with_cluster_value.keys()

dict_keys(['total_outstanding_orders', 'total_onshift_dashers', 'availabe_dashers_onshift'])

## Splitting training and testing to avoid data leeakge 

In [14]:
data_cleaning_no_categorical_na = data_cleaning_no_categorical_na.sort_values('created_at')
training_split = int(np.ceil(0.8*data_cleaning_no_categorical_na.shape[0]))
training_data = data_cleaning_no_categorical_na[:training_split] 
testing_data = data_cleaning_no_categorical_na[training_split:]

In [15]:
curated_training_data = DataCleaner.clean_conflict_category(training_data.copy())

In [16]:
training_data.loc[training_data['store_id'].eq(4210)]['market_id'].value_counts()

market_id
2.0             370
1.0               4
4.0               3
5.0               2
not informed      2
6.0               2
3.0               1
Name: count, dtype: int64

In [17]:
curated_training_data.loc[curated_training_data['store_id'].eq(4210)]['market_id'].value_counts()

market_id
2.0    384
Name: count, dtype: int64

## Input missing data

In [102]:
whole_data['time_of_day'].value_counts(normalize=True)*100

time_of_day
Afternoon    65.823369
Evening      19.048995
Night        14.404519
Morning       0.723116
Name: proportion, dtype: float64

## Correct food_category and market_id based on store

In [18]:
whole_data_market_id = whole_data\
        .groupby(['market_id','store_id'],as_index=False)\
        .agg(first_occurence=('created_at','min'), count_subtotal=('subtotal','count'), avg_subtotal=('subtotal','mean'), median_subtotal=('subtotal','median'))\
        .sort_values(['store_id','count_subtotal'],ascending=[False, False])

whole_data_primary_category = whole_data\
        .groupby(['store_primary_category','store_id'],as_index=False)\
        .agg(first_occurence=('created_at','min'), count_subtotal=('subtotal','count'), avg_subtotal=('subtotal','mean'), median_subtotal=('subtotal','median'))\
        .sort_values(['store_id', 'count_subtotal'],ascending=[False, False])

In [19]:
# display(whole_data_market_id.loc[whole_data_market_id['store_id'].eq(314)])
# display(whole_data_primary_category.loc[whole_data_primary_category['store_id'].eq(314)])

In [20]:
whole_data_market_id_maxs = whole_data_market_id.groupby('store_id', as_index=False).agg(count_subtotal=('count_subtotal','max')) 
whole_data_market_id_filtered = whole_data_market_id.merge(whole_data_market_id_maxs, how='inner', on=['store_id','count_subtotal'])
whole_data_market_id_mins = whole_data_market_id_filtered.groupby('store_id', as_index=False).agg(first_occurence=('first_occurence','min')) 
whole_data_market_id_unique = whole_data_market_id_filtered.merge(whole_data_market_id_mins, how='inner', on=['store_id','first_occurence'])

whole_data_primary_category_maxs = whole_data_primary_category.groupby('store_id', as_index=False).agg(count_subtotal=('count_subtotal','max')) 
whole_data_primary_category_filtered = whole_data_primary_category.merge(whole_data_primary_category_maxs, how='inner', on=['store_id','count_subtotal'])
whole_data_primary_category_mins = whole_data_primary_category_filtered.groupby('store_id', as_index=False).agg(first_occurence=('first_occurence','min')) 
whole_data_primary_category_unique = whole_data_primary_category_filtered.merge(whole_data_primary_category_mins, how='inner', on=['store_id','first_occurence'])

In [21]:
display(whole_data_market_id_filtered.loc[whole_data_market_id_filtered['store_id'].eq(314)])
display(whole_data_primary_category_filtered.loc[whole_data_primary_category_filtered['store_id'].eq(314)])

,market_id,store_id,first_occurence,count_subtotal,avg_subtotal,median_subtotal
6488,2.0,314,2015-01-21 17:58:11,786,1795.263359,1548.5


,store_primary_category,store_id,first_occurence,count_subtotal,avg_subtotal,median_subtotal
6490,mexican,314,2015-01-21 17:58:11,775,1782.268387,1548.0


In [22]:
whole_data.shape

(197202, 21)

In [23]:
whole_data_market_id_unique

,market_id,store_id,first_occurence,count_subtotal,avg_subtotal,median_subtotal
0,3.0,6987,2015-01-25 04:58:46,2,4350.000000,4350.0
1,4.0,6986,2015-01-21 20:25:49,36,1179.888889,998.0
2,1.0,6985,2015-01-31 03:45:03,2,1085.000000,1085.0
3,6.0,6984,2015-01-31 02:49:56,4,3087.500000,3340.0
4,1.0,6983,2015-01-21 22:01:56,24,2472.000000,2451.5
...,...,...,...,...,...,...
6736,6.0,5,2015-01-26 18:29:58,1,2256.000000,2256.0
6737,1.0,4,2015-01-22 02:26:28,130,1592.046154,1303.0
6738,3.0,3,2015-02-05 03:58:47,3,4133.333333,4200.0
6739,1.0,2,2015-01-22 20:40:45,5,3764.400000,3420.0


In [24]:
whole_data_test = whole_data.drop(['market_id', 'store_primary_category'],axis=1)
whole_data_test = \
        whole_data_test\
            .merge(whole_data_primary_category_unique[['store_id','store_primary_category']], on='store_id')\
            .merge(whole_data_market_id_unique[['store_id','market_id']], on='store_id')
# whole_data_test = curated_data.copy()

In [25]:
print(whole_data.shape)
print(whole_data_test.shape)

(197202, 21)
(197202, 21)


In [26]:
whole_data.loc[whole_data['store_id'].eq(791)]['store_primary_category'].value_counts()

store_primary_category
not informed    180
asian             1
sushi             1
thai              1
greek             1
Name: count, dtype: int64

In [27]:
whole_data_test.loc[whole_data_test['store_id'].eq(791)]['store_primary_category'].value_counts()

store_primary_category
not informed    184
Name: count, dtype: int64

In [28]:
display(whole_data_primary_category_filtered.loc[whole_data_primary_category_filtered['store_id'].eq(2387)])
whole_data_primary_category_unique.loc[whole_data_primary_category_unique['store_id'].eq(2387)]

,store_primary_category,store_id,first_occurence,count_subtotal,avg_subtotal,median_subtotal
4484,burger,2387,2015-01-22 02:47:51,1,3690.0,3690.0
4485,korean,2387,2015-02-07 23:36:57,1,2790.0,2790.0
4486,mediterranean,2387,2015-01-25 03:44:16,1,1395.0,1395.0


,store_primary_category,store_id,first_occurence,count_subtotal,avg_subtotal,median_subtotal
4443,burger,2387,2015-01-22 02:47:51,1,3690.0,3690.0


In [29]:
display(whole_data_market_id_filtered.loc[whole_data_market_id_filtered['store_id'].eq(104)])
whole_data_market_id_unique.loc[whole_data_market_id_unique['store_id'].eq(104)]

,market_id,store_id,first_occurence,count_subtotal,avg_subtotal,median_subtotal
6692,4.0,104,2015-01-28 19:23:16,1,2700.0,2700.0
6693,5.0,104,2015-02-11 23:22:02,1,1375.0,1375.0
6694,6.0,104,2015-02-06 23:08:21,1,1375.0,1375.0


,market_id,store_id,first_occurence,count_subtotal,avg_subtotal,median_subtotal
6641,4.0,104,2015-01-28 19:23:16,1,2700.0,2700.0


In [30]:
whole_data_test.loc[whole_data_test['order_protocol'].isna()]

,created_at,actual_delivery_time,store_id,order_protocol,total_items,subtotal,num_distinct_items,total_onshift_dashers,total_outstanding_orders,estimated_order_place_duration,...,weekday,weekend,hour,time_of_day,estimated_delivery_time,delivery_time,delivery_time_hours,availabe_dashers_onshift,store_primary_category,market_id


In [31]:
whole_data_test.head()

,created_at,actual_delivery_time,store_id,order_protocol,total_items,subtotal,num_distinct_items,total_onshift_dashers,total_outstanding_orders,estimated_order_place_duration,...,weekday,weekend,hour,time_of_day,estimated_delivery_time,delivery_time,delivery_time_hours,availabe_dashers_onshift,store_primary_category,market_id
0,2015-01-21 15:22:03,2015-01-21 16:17:43,2966,4.0,2,1058,2,2.0,1.0,251,...,2,0,15,Afternoon,714.0,3340.0,0.927778,1.0,dessert,1.0
1,2015-01-22 18:13:58,2015-01-22 18:42:44,2966,4.0,2,1598,1,10.0,8.0,251,...,3,0,18,Evening,996.0,1726.0,0.479444,2.0,dessert,1.0
2,2015-01-22 18:57:55,2015-01-22 19:19:14,2966,4.0,1,599,1,17.0,12.0,251,...,3,0,18,Evening,959.0,1279.0,0.355278,5.0,dessert,1.0
3,2015-01-22 19:22:16,2015-01-22 19:55:00,2966,4.0,2,198,1,19.0,17.0,251,...,3,0,19,Evening,963.0,1964.0,0.545556,2.0,dessert,1.0
4,2015-01-24 00:46:42,2015-01-24 01:28:12,2966,4.0,1,1048,1,24.0,18.0,251,...,5,1,0,Afternoon,679.0,2490.0,0.691667,5.0,dessert,1.0


## Input missing data

In [32]:
x = whole_data[['store_id','store_primary_category']].value_counts().sort_index().to_frame().reset_index()
x['store_id'].value_counts().head()

store_id
6865    26
314     23
1311    23
1686    22
3937    22
Name: count, dtype: int64

In [33]:
whole_data.groupby(['market_id','time_of_day'],as_index=False).agg(median_value = ('total_onshift_dashers','median'),).pivot(index='market_id',columns='time_of_day', values='median_value')

time_of_day,Afternoon,Evening,Morning,Night
market_id,,,,
1.0,24.0,14.0,6.0,14.0
2.0,75.0,39.0,18.0,33.0
3.0,19.0,10.0,5.0,11.0
4.0,72.0,47.0,11.0,30.0
5.0,24.0,15.0,9.0,14.0
6.0,47.0,25.0,12.0,21.0
not informed,47.5,24.0,6.5,21.0


In [34]:
fillna_datahandler.fill_with_cluster_value.keys()

dict_keys(['total_outstanding_orders', 'total_onshift_dashers', 'availabe_dashers_onshift'])

In [35]:
x = whole_data_test[['store_id','store_primary_category']].value_counts().sort_index().to_frame().reset_index()
x['store_id'].value_counts().head()

store_id
1       1
4645    1
4671    1
4670    1
4668    1
Name: count, dtype: int64

In [36]:
y = whole_data_test.loc[whole_data_test['store_id'].isin([314]),['store_id','market_id','store_primary_category']]
y = y.drop_duplicates()
y.sort_values(['market_id','store_id']).head(50)

,store_id,market_id,store_primary_category
10271,314,2.0,mexican


In [37]:
whole_data_test

,created_at,actual_delivery_time,store_id,order_protocol,total_items,subtotal,num_distinct_items,total_onshift_dashers,total_outstanding_orders,estimated_order_place_duration,...,weekday,weekend,hour,time_of_day,estimated_delivery_time,delivery_time,delivery_time_hours,availabe_dashers_onshift,store_primary_category,market_id
0,2015-01-21 15:22:03,2015-01-21 16:17:43,2966,4.0,2,1058,2,2.0,1.0,251,...,2,0,15,Afternoon,714.0,3340.0,0.927778,1.0,dessert,1.0
1,2015-01-22 18:13:58,2015-01-22 18:42:44,2966,4.0,2,1598,1,10.0,8.0,251,...,3,0,18,Evening,996.0,1726.0,0.479444,2.0,dessert,1.0
2,2015-01-22 18:57:55,2015-01-22 19:19:14,2966,4.0,1,599,1,17.0,12.0,251,...,3,0,18,Evening,959.0,1279.0,0.355278,5.0,dessert,1.0
3,2015-01-22 19:22:16,2015-01-22 19:55:00,2966,4.0,2,198,1,19.0,17.0,251,...,3,0,19,Evening,963.0,1964.0,0.545556,2.0,dessert,1.0
4,2015-01-24 00:46:42,2015-01-24 01:28:12,2966,4.0,1,1048,1,24.0,18.0,251,...,5,1,0,Afternoon,679.0,2490.0,0.691667,5.0,dessert,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197197,2015-02-18 02:22:11,2015-02-18 03:06:27,482,1.0,2,2295,2,9.0,6.0,446,...,2,0,2,Afternoon,795.0,2656.0,0.737778,3.0,japanese,1.0
197198,2015-02-18 02:37:52,2015-02-18 04:24:14,2640,1.0,2,2250,2,109.0,103.0,446,...,2,0,2,Afternoon,1014.0,6382.0,1.772778,38.0,not informed,2.0
197199,2015-02-18 02:39:35,2015-02-18 03:23:53,5913,1.0,4,4396,4,109.0,103.0,446,...,2,0,2,Afternoon,1180.0,2658.0,0.738333,38.0,barbecue,2.0
197200,2015-02-18 04:59:36,2015-02-18 05:26:05,5913,1.0,1,1798,1,49.0,52.0,446,...,2,0,4,Afternoon,1158.0,1589.0,0.441389,3.0,barbecue,2.0


In [38]:
x = whole_data_test.groupby(['market_id','time_of_day'],as_index=False).agg(median_value = ('total_onshift_dashers','median'))
x.columns[:-1]

Index(['market_id', 'time_of_day'], dtype='object')

In [39]:
print(f'''Overall median total_onshift_dashers = {whole_data_test[['total_onshift_dashers']].median().values[0]}''')
display(whole_data_test.groupby(['market_id','time_of_day'],as_index=False).agg(median_value = ('total_onshift_dashers','median'),).pivot(index='market_id',columns='time_of_day', values='median_value')[['Morning', 'Afternoon', 'Evening', 'Night']])

print(f'''Overall median availabe_dashers_onshift = {whole_data_test[['availabe_dashers_onshift']].median().values[0]}''')
display(whole_data_test.groupby(['market_id','time_of_day'],as_index=False).agg(median_value = ('availabe_dashers_onshift','median'),).pivot(index='market_id',columns='time_of_day', values='median_value')[['Morning', 'Afternoon', 'Evening', 'Night']])

print(f'''Overall median total_outstanding_orders = {whole_data_test[['total_outstanding_orders']].median().values[0]}''')
display(whole_data_test.groupby(['market_id','time_of_day'],as_index=False).agg(median_value = ('total_outstanding_orders','median'),).pivot(index='market_id',columns='time_of_day', values='median_value')[['Morning', 'Afternoon', 'Evening', 'Night']])


Overall median total_onshift_dashers = 37.0


time_of_day,Morning,Afternoon,Evening,Night
market_id,,,,
1.0,5.0,23.0,14.0,13.0
2.0,19.0,76.0,39.0,33.0
3.0,5.0,18.0,9.0,11.0
4.0,11.0,73.0,47.0,30.0
5.0,8.0,24.0,15.0,14.0
6.0,NaN,9.0,15.0,NaN
not informed,NaN,35.0,19.0,8.5


Overall median availabe_dashers_onshift = 1.0


time_of_day,Morning,Afternoon,Evening,Night
market_id,,,,
1.0,0.0,0.0,0.0,0.0
2.0,0.0,4.0,1.0,1.0
3.0,0.0,0.0,0.0,1.0
4.0,1.0,4.0,1.0,3.0
5.0,2.0,4.0,2.0,3.0
6.0,NaN,0.0,0.0,NaN
not informed,NaN,8.0,0.0,0.0


Overall median total_outstanding_orders = 41.0


time_of_day,Morning,Afternoon,Evening,Night
market_id,,,,
1.0,5.0,29.0,15.0,13.0
2.0,23.0,96.0,50.0,37.0
3.0,4.0,18.0,9.0,9.0
4.0,9.0,101.0,57.0,28.0
5.0,7.0,26.0,14.0,11.0
6.0,NaN,12.0,20.0,NaN
not informed,NaN,48.0,14.0,10.5


In [40]:
fillna_datahandler.fill_with_cluster_value['total_onshift_dashers'].pivot(index='market_id',columns='time_of_day', values='median_value')[['Morning', 'Afternoon', 'Evening', 'Night']]

time_of_day,Morning,Afternoon,Evening,Night
market_id,,,,
1.0,5.0,23.0,14.0,13.0
2.0,19.0,76.0,39.0,33.0
3.0,5.0,18.0,9.0,11.0
4.0,11.0,73.0,47.0,30.0
5.0,8.0,24.0,15.0,14.0
6.0,NaN,9.0,15.0,NaN
not informed,NaN,35.0,19.0,8.5


In [41]:
fillna_datahandler.fill_with_cluster_value['total_onshift_dashers'].pivot(index='market_id',columns='time_of_day', values='median_value')[['Morning', 'Afternoon', 'Evening', 'Night']]

time_of_day,Morning,Afternoon,Evening,Night
market_id,,,,
1.0,5.0,23.0,14.0,13.0
2.0,19.0,76.0,39.0,33.0
3.0,5.0,18.0,9.0,11.0
4.0,11.0,73.0,47.0,30.0
5.0,8.0,24.0,15.0,14.0
6.0,NaN,9.0,15.0,NaN
not informed,NaN,35.0,19.0,8.5


In [374]:
whole_data_test['market_id'].value_counts()

market_id
2.0             54966
4.0             47628
1.0             37845
3.0             23212
5.0             18144
6.0             14498
not informed      909
Name: count, dtype: int64

In [ ]:
## Adding the trivial varialbes

#target

#temporal variables 


#
